## 파이토치 기본
### 파이토치 공식 튜토리얼
- https://tutorials.pytorch.kr/

### 파이토치 패키지(API)
- torch : 메인 네임스페이스. 텐서(기본담위) 등 수학함수 포함. Numpy와 유사한 구조
- torch.autograd : 자동미분을 위한 함수들 포함. 컨텍스트 매니저, 기반클래스 포함
- torch.nn / torch.nn.functional : neuralnetwork. 신경망 구축위한 데이터 구조, 레이어등이 정의되어 있는 모듈. ReLU 등도 포함
- torch.optim : SGD(확률적 경사 하강법) 중심 모델 파라미터 최적화 알고리즘 표함
- torch.utils : SGD 반복 연산 시 배치용 유틸리티 함수 포함
- 

## 텐서
- 파이토치 기본단위
    - 1차원 배열 - 벡터
    - 2차원 배열 - 행렬(매트릭스)
    - 3차원 배열 = 텐서
    
<img src='../image/ml17.png' width='600'>
<img src='../image/ml18.png' width='600'>

In [6]:
import torch
import numpy as np

In [7]:
t1 = torch.tensor([1.0, 2.0, 3.0])

In [9]:
n1 = np.array([1.0, 2.0, 3.0])
n1

array([1., 2., 3.])

In [16]:
## numpy에서 1, 0으로 초기화 되는
n2 = np.ones([2, 3])
n2

array([[1., 1., 1.],
       [1., 1., 1.]])

In [12]:
n3 = np.zeros([2, 3])
n3

array([[0., 0., 0.],
       [0., 0., 0.]])

In [13]:
t2 = torch.ones(2, 3)
t2

tensor([[1., 1., 1.],
        [1., 1., 1.]])

In [14]:
t3 = torch.zeros(2, 3)
t3

tensor([[0., 0., 0.],
        [0., 0., 0.]])

In [15]:
# numpy로 만든 배열을 텐서로 변환
t4 = torch.from_numpy(n3)
t4

tensor([[0., 0., 0.],
        [0., 0., 0.]], dtype=torch.float64)